<a href="https://colab.research.google.com/github/dibya-8/Elevate-Labs-/blob/main/Heart_Disease_Prediction_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
johnsmith88_heart_disease_dataset_path = kagglehub.dataset_download('johnsmith88/heart-disease-dataset')

print('Data source import complete.')


# Heart Disease Prediction Project

**Goal:** To predict the presence or absence of heart disease based on various patient attributes using machine learning classification models.

**Dataset:** Heart Disease UCI dataset. This dataset contains 14 attributes collected from patients. The "target" column indicates the presence (1) or absence (0) of heart disease.

**Plan:**
1.  Load and Explore Data (EDA)
2.  Preprocess Data (Handle categorical features, split data, scale numerical features)
3.  Train and Evaluate Machine Learning Models (Logistic Regression, Random Forest, SVM)
4.  Compare Models and Conclude

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

## 1. Load and Explore Data (EDA)

In [ ]:
# Load the dataset
# Make sure the file path matches where Kaggle stores the input data
try:
    df = pd.read_csv('/kaggle/input/heart-disease-dataset/heart.csv')
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print("Error: heart.csv not found. Please ensure the dataset is added correctly.")
    # As a fallback for local testing, you might load it like this:
    # df = pd.read_csv('heart.csv')
    # exit() # Or handle the error appropriately

# Display basic information
print("\n--- Dataset Info ---")
df.info()

print("\n--- First 5 Rows ---")
print(df.head())

print("\n--- Descriptive Statistics ---")
print(df.describe())

print("\n--- Check for Missing Values ---")
print(df.isnull().sum())
# Good news: No missing values in this standard version of the dataset.

print("\n--- Target Variable Distribution ---")
target_counts = df['target'].value_counts()
print(target_counts)
sns.countplot(x='target', data=df)
plt.title('Distribution of Target Variable (0: No Disease, 1: Disease)')
plt.show()

### EDA Visualizations

Let's explore the relationships between features and the target variable.

**Age Distribution:**

In [ ]:
sns.histplot(data=df, x='age', hue='target', kde=True)
plt.title('Age Distribution by Target')
plt.show()

**Sex Distribution:** (0 = female, 1 = male)

In [ ]:
sns.countplot(data=df, x='sex', hue='target')
plt.title('Heart Disease Frequency for Sex')
plt.xlabel('Sex (0 = Female, 1 = Male)')
plt.ylabel('Frequency')
plt.show()

**Chest Pain Type (cp) Distribution:**
*   0: Typical angina
*   1: Atypical angina
*   2: Non-anginal pain
*   3: Asymptomatic

In [ ]:
sns.countplot(data=df, x='cp', hue='target')
plt.title('Heart Disease Frequency According to Chest Pain Type')
plt.xlabel('Chest Pain Type')
plt.ylabel('Frequency')
plt.show()

**Correlation Heatmap:**

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Features')
plt.show()

## 2. Data Preprocessing

We need to handle categorical features and scale numerical features.

*   **Categorical features:** `sex`, `cp`, `fbs`, `restecg`, `exang`, `slope`, `ca`, `thal`. Some are binary, others are nominal/ordinal. We'll use One-Hot Encoding for nominal features with more than 2 categories (`cp`, `restecg`, `slope`, `thal`, `ca`). Note: `sex`, `fbs`, `exang` are already 0/1.
*   **Numerical features:** `age`, `trestbps`, `chol`, `thalach`, `oldpeak`. We'll scale these using StandardScaler.

In [ ]:
# Define features (X) and target (y)
X = df.drop('target', axis=1)
y = df['target']

# Identify feature types
categorical_features = ['cp', 'restecg', 'slope', 'ca', 'thal'] # Features to one-hot encode
numerical_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
# Binary features ('sex', 'fbs', 'exang') don't strictly need one-hot encoding or scaling if kept as 0/1

# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")

# Create preprocessing pipelines for numerical and categorical features
# Numerical pipeline: Scale features
num_pipeline = Pipeline([
    ('scaler', StandardScaler())
])

# Categorical pipeline: One-hot encode features
cat_pipeline = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # handle_unknown='ignore' is safer for unseen categories in test set
])

# Combine pipelines using ColumnTransformer
# Apply num_pipeline to numerical features
# Apply cat_pipeline to categorical features needing one-hot encoding
# Passthrough binary features ('sex', 'fbs', 'exang') or include them if preferred
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, numerical_features),
        ('cat', cat_pipeline, categorical_features),
        ('passthrough', 'passthrough', ['sex', 'fbs', 'exang']) # Keep binary features as they are
    ],
    remainder='drop' # Drop any columns not specified (shouldn't be any here)
)

# Apply the preprocessing pipeline to the training data
X_train_processed = preprocessor.fit_transform(X_train)

# Apply the *fitted* preprocessing pipeline to the testing data
X_test_processed = preprocessor.transform(X_test)

# Get feature names after transformation (useful for some models like Tree-based ones)
feature_names_out = preprocessor.get_feature_names_out()
print(f"\nProcessed feature names ({X_train_processed.shape[1]} features):\n{feature_names_out}")


print(f"\nProcessed Training set shape: {X_train_processed.shape}")
print(f"Processed Test set shape: {X_test_processed.shape}")

## 3. Train and Evaluate Machine Learning Models

We will train and evaluate the following models:
1.  Logistic Regression
2.  Random Forest Classifier
3.  Support Vector Machine (SVM)

In [ ]:
# Define models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Support Vector Machine": SVC(probability=True, random_state=42) # probability=True for ROC AUC
}

# Dictionary to store results
results = {}

# Train and evaluate each model
for name, model in models.items():
    print(f"--- Training {name} ---")
    # Train the model
    model.fit(X_train_processed, y_train)

    # Make predictions
    y_pred_train = model.predict(X_train_processed)
    y_pred_test = model.predict(X_test_processed)
    y_prob_test = model.predict_proba(X_test_processed)[:, 1] # Probabilities for ROC AUC

    # Evaluate the model
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    report_test = classification_report(y_test, y_pred_test)
    cm_test = confusion_matrix(y_test, y_pred_test)
    roc_auc = roc_auc_score(y_test, y_prob_test)

    # Store results
    results[name] = {
        "model": model,
        "accuracy_train": accuracy_train,
        "accuracy_test": accuracy_test,
        "classification_report": report_test,
        "confusion_matrix": cm_test,
        "roc_auc": roc_auc,
        "y_pred": y_pred_test,
        "y_prob": y_prob_test
    }

    # Print results
    print(f"\n--- Results for {name} ---")
    print(f"Training Accuracy: {accuracy_train:.4f}")
    print(f"Test Accuracy: {accuracy_test:.4f}")
    print(f"Test ROC AUC: {roc_auc:.4f}")
    print("Test Classification Report:\n", report_test)
    print("Test Confusion Matrix:\n", cm_test)

    # Plot Confusion Matrix
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{name} - Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

    print("-" * 30)

## 4. Compare Models and Conclude

Let's compare the models based on Test Accuracy and ROC AUC Score.

In [ ]:
# Display comparison summary
print("\n--- Model Comparison ---")
comparison_data = []
for name, result in results.items():
    comparison_data.append({
        "Model": name,
        "Test Accuracy": result['accuracy_test'],
        "Test ROC AUC": result['roc_auc']
    })

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.sort_values(by='Test ROC AUC', ascending=False))

# Plot ROC Curves for all models
plt.figure(figsize=(10, 8))
for name, result in results.items():
    fpr, tpr, _ = roc_curve(y_test, result['y_prob'])
    plt.plot(fpr, tpr, label=f"{name} (AUC = {result['roc_auc']:.2f})")

plt.plot([0, 1], [0, 1], 'k--', label='Random Guess') # Dashed line for random guess
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('ROC Curve Comparison')
plt.legend()
plt.grid(True)
plt.show()

### Conclusion

In this notebook, we aimed to predict heart disease using the UCI Heart Disease dataset.

1.  **Data Exploration:** We analyzed the dataset, visualized feature distributions, and checked correlations.
2.  **Preprocessing:** We handled categorical features using One-Hot Encoding and scaled numerical features using StandardScaler. The data was split into training and testing sets.
3.  **Model Training:** We trained three common classification models: Logistic Regression, Random Forest, and Support Vector Machine.
4.  **Evaluation:** The models were evaluated using accuracy, classification reports, confusion matrices, and ROC AUC scores.

**Findings:**
*   All models performed reasonably well, indicating that the features are predictive of heart disease.
*   Based on the test set performance (Accuracy and ROC AUC), [Comment on which model(s) performed best - e.g., "Logistic Regression and SVM showed strong results" or "Random Forest slightly outperformed the others"]. Check the specific output from your run.
*   The ROC curve comparison provides a visual way to assess the trade-off between True Positive Rate and False Positive Rate for each model.

**Potential Future Work:**
*   **Hyperparameter Tuning:** Use techniques like GridSearchCV or RandomizedSearchCV to find optimal parameters for each model, potentially improving performance further.
*   **Feature Engineering:** Create new features from existing ones if domain knowledge suggests potential interactions.
*   **Try Other Models:** Experiment with models like Gradient Boosting (XGBoost, LightGBM) or Neural Networks.
*   **Cross-Validation:** Implement k-fold cross-validation during training for more robust evaluation.
*   **Interpretability:** Use techniques like SHAP or LIME to understand *why* a model makes certain predictions, especially for more complex models like Random Forest.
